In [1]:
import nltk
import xml.etree.cElementTree as et
import heapq
import numpy as np #matrix
from numpy import dot #dot product used to calcualte cosine similarity
from numpy.linalg import norm #normalisation used to calculate cosine similarity
from nltk.tokenize import RegexpTokenizer #used to tokenize into words and remove punctuation
from nltk.corpus import stopwords

from tqdm import tqdm #extra function to monitor lengthy calls 
import time

getListOfFiles is used to retrieve the names of each file within docs/, please note for this to work both folders will have to be in the same directory as this jupiter notebook

In [2]:
import os

def getListOfFiles(dirName): #fuction used to retrieve parse the document to extract
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles
dirName = 'docs/'; #directory conatining all the documents
listOfFiles = getListOfFiles(dirName)
#print (listOfFiles)

extractData is used to extract the data in XML's <raw> tag

In [3]:
def extractData(string):
 root = et.parse(string).getroot()   
 for raw in root.iter('raw'): #extracts paragraphs of information from raw tags
   data = raw.text
 return(data)

tokenizer function that returns a tokenized paragraph whilst removing punctuation

In [4]:
tokenizer = RegexpTokenizer(r'\w+')

def tokenise(string): 
 tokens_document = tokenizer.tokenize(string)
 return tokens_document

#text = tokenise(text)
#print(text)

The following three functions are used to handle the casefolding, stopwordremoval and stemming of text

In [5]:
def caseFold (list): #function which handles casefolding
 casefoldedList = [word.casefold() for word in list]
    
 return casefoldedList   

#text = caseFold(text)
#print(text)

In [6]:
def stopWordRemoval(list): #stopwords is used from nltk and stopWordRemoval is called to remove stop words from the passed document
 newlist = []
 stopWords = set(stopwords.words('english'))
    
 for word in list:
    if word not in stopWords:
        newlist.append(word)

 return newlist

In [7]:
from nltk.stem import PorterStemmer
def stemm(list): #function which handles a documents stemming
  Stem = PorterStemmer() 
  stemmList = [Stem.stem(word) for word in list] 
  return stemmList

#text = stemm(text)
#print(text)

cleaning is the function that handles tokenization case-folding, stop-word removal and stemming

In [8]:
def cleaning(toClean):
 text_token = tokenise(toClean)
 text_cf = caseFold(text_token)
 text_sWR = stopWordRemoval(text_cf)
 final_text = stemm(text_sWR)
 
 return(final_text)   

#text = cleaning(text)
#print(text)

Below, the extraction of each document, tokenization and cleaning takes places and each fully cleaned document is added doc_list 

tqdm is used to monitor the status of of what document has been added to the list

In [9]:
doc_list = []

LENGTH = len(listOfFiles)
pbar = tqdm(total=LENGTH)# Number of iterations required to fill pbar
for doc_name in listOfFiles:   
 text = extractData(doc_name) #handles data extraction
 text = cleaning(text) #handles case-foling, stop-word removal, and stemming
 doc_list.append(text)
 pbar.update(n=1) # Increments counter

#print(doc_list)


100%|██████████| 331/331 [00:20<00:00, 76.83it/s]

Below what has already been implemented is used to again to obtain the list of all possible queries

In [10]:
query_list = [] 
dirName = 'queries/'; #directory of queries 
listOfFilesQ = getListOfFiles(dirName)
listOfFilesQ.pop(0)

for query_name in listOfFilesQ:   
 text = extractData(query_name)
 text = cleaning(text)  
 query_list.append(text)

print(query_list)


[['fabric', 'music', 'instrument'], ['famou', 'german', 'poetri'], ['romantic'], ['univers', 'edinburgh', 'research'], ['bridg', 'construct'], ['walk', 'fame', 'star'], ['scientist', 'work', 'atom', 'bomb'], ['invent', 'internet'], ['earli', 'telecommun', 'method'], ['explor', 'south', 'pole'], ['famou', 'member', 'royal', 'navi'], ['nobel', 'prize', 'win', 'invent'], ['south', 'america'], ['edward', 'teller', 'mari', 'curi'], ['comput', 'languag', 'program', 'artifici', 'intellig'], ['william', 'hearst', 'movi'], ['captain', 'jame', 'cook', 'becom', 'explor'], ['grace', 'hopper', 'get', 'famou'], ['comput', 'astronomi'], ['wwii', 'aircraft'], ['literari', 'critic', 'thoma', 'moor'], ['nazi', 'confisc', 'destroy', 'art', 'literatur'], ['modern', 'age', 'english', 'literatur'], ['modern', 'physiolog'], ['roman', 'empir'], ['scientist', 'contribut', 'photosynthesi'], ['aviat', 'pioneer', 'public'], ['gutenberg', 'bibl'], ['religi', 'belief', 'scientist', 'explor'], ['carl', 'friedrich', 

The query_list is added to the new_doc_list so that after for cosine similarity the matrices have the same length

In [11]:
doc_list.extend(query_list)

CalWordFreq takes a list and sorts it in the order of which term is most frequent, this is used during the calculations of tf*idf

In [13]:
def CalWordFreq(list):
 wordfreq = {}
 most_freq = []
 for sentence in list:
     tokens = sentence
     for word in tokens:
       if word not in wordfreq.keys():
          wordfreq[word] = 1
       else:
          wordfreq[word] += 1      
        
 sort_orders = sorted(wordfreq.items(), key=lambda x: x[1], reverse=True)
 for i in sort_orders:
    most_freq.append(i[0])
 return(most_freq)   

print(CalWordFreq(doc_list))

['first', 'work', 'also', 'year', 'one', 'becam', 'univers', 'use', 'born', 'time', 'known', 'studi', 'publish', 'later', 'new', 'de', 'comput', 'interest', '1', 'yovisto', 'develop', 'mathemat', 'made', 'scienc', 'howev', 'may', 'lectur', '2', 'two', 'world', 'theori', 'german', 'well', 'engin', 'school', 'found', 'age', 'research', 'earli', 'began', 'mani', 'learn', 'father', 'receiv', 'scientist', 'john', 'famou', 'system', 'includ', 'life', 'would', 'start', 'could', 'book', 'design', 'sever', 'even', 'call', 'name', 'physic', 'professor', '3', 'societi', 'continu', 'war', 'experi', 'die', 'famili', 'becom', 'import', 'american', 'french', 'state', 'invent', 'order', 'part', 'follow', 'histori', 'make', 'germani', 'scientif', 'return', 'move', 'natur', 'great', 'three', 'public', 'educ', 'believ', 'day', 'success', 'berlin', 'result', 'space', 'pari', 'institut', 'video', 'produc', 'creat', 'took', 'abl', 'machin', 'period', 'discov', 'colleg', 'best', 'young', 'man', 'still', 'pas

For the calculation of idf, if a work appears multiple times within a document appears gets increamented the forumla for idf is used and this value is returned to be calculated with the tf values

In [14]:
def idfValues(freq_list, orig_list):
 idf = {}
 for word in freq_list:
     appears = 0
     for document in orig_list:
         if word in document:
             appears += 1
     idf[word] = np.log(len(orig_list)/(1 + appears)) #IDF: (Total number of documents)/(Number of documents containing the word

 return idf

#idf_values = idfValues(most_freq,final_text)

TF = (Number of times term t appears in a document) / (Total number of terms in the document).
The tfvalues are calulated using a list sent_tf_vector and the individual scores of TF are appended to the function

In [15]:
def tfValues(freq_list, orig_list):
 tf = {}
 for word in freq_list:
    sent_tf_vector = []
    for document in orig_list:
        doc_freq = 0
        for word_doc in document:
            if word == word_doc:
                  doc_freq += 1
        word_tf = doc_freq/len(document) #TF = (Number of times term t appears in a document) / (Total number of terms in the document).
        sent_tf_vector.append(word_tf)
        tf[word] = sent_tf_vector
 return tf       
    
#tf_values = tfValues(most_freq,final_text)

tfidf is calculated by multiply the values given by both tf and idf

In [16]:
def tfidf(valuesTf, valuesIdf):
 tfidf_values = []
 for word in valuesTf.keys():
     tfidf_sentences = []
     for tf_sentence in valuesTf[word]:
         tf_idf_score = tf_sentence * valuesIdf[word]
         tfidf_sentences.append(tf_idf_score)
     tfidf_values.append(tfidf_sentences)
 tf_idf_model = np.asarray(tfidf_values) 
 tf_idf_model = np.transpose(tf_idf_model)
    
 return tf_idf_model

#print(tfidf(tf_values,idf_values))

The VectorSpaceModel function utilizes all the functions that have been implemented  to calculate the final Term by document matrix.

In [17]:
def VectorSpaceModel(DocByTerm):

   most_freq_text = CalWordFreq(DocByTerm) #returns a list of the most frequent words

   idf_values = idfValues(most_freq_text,DocByTerm)
   tf_values = tfValues(most_freq_text,DocByTerm)
   tfidf_values = tfidf(tf_values,idf_values)
   return tfidf_values
    

Finally the term by document matrix is calculated with the rows signifying the documents (upto 331) and the queries up to 336

In [18]:
TermByDocument = VectorSpaceModel(doc_list)
print(TermByDocument)

[[0.         0.00123718 0.00181001 ... 0.         0.         0.        ]
 [0.00281198 0.00128136 0.00093733 ... 0.         0.         0.        ]
 [0.00132205 0.00120486 0.00044068 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.05905166 0.         0.         ... 0.         0.         0.        ]]


In [19]:
num_rows, num_cols = TermByDocument.shape #last 35 rows are the queries 
print ("Rows:", num_rows, "Cols:", num_cols)

Rows: 366 Cols: 13539


Finally the cosine similarity function is implemented via its formule dot(doc,query)/norm(doc) * norm(query)

The top 10 scores have been printed to screen but the ranked_list contains all the scores for the 331 documents

In [20]:
for j in range (332, 366):  #go over every query
    list = []
    ranked_list = []
    all_ranked_lists = []
    
    for i in range(331): #go over every document
        cosine_sim = dot(TermByDocument[i] , TermByDocument[j]) / norm(TermByDocument[i]) * norm(TermByDocument[j])
        list.append(cosine_sim)
    
    for score in enumerate(list): #adds a value to each score that keeps track of which document has which score
        ranked_list.append(score)    
    
    ranked_list.sort(key=lambda x:x[1], reverse=True) #sorts the list by greatest score
    print("first value: Document, second value:Score")
    print("Ranked list for query "+str(j-331))
    print(ranked_list[:10]) #prints top 10 scores
    print()

#print(all_ranked_lists) 

first value: Document, second value:Score
Ranked list for query 1
[(146, 0.17220632416374518), (282, 0.11071333162022277), (148, 0.09157763850166976), (290, 0.07416594790543331), (133, 0.07155201069495618), (292, 0.06183136979725583), (22, 0.05472489302827605), (135, 0.04855117091021798), (1, 0.0465079112211394), (138, 0.040149150915555265)]

first value: Document, second value:Score
Ranked list for query 2
[(282, 0.8590726012252736), (290, 0.6358106762275691), (104, 0.4317236609959397), (151, 0.4206645770499912), (146, 0.4117408736485981), (317, 0.31218020528931834), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)]

first value: Document, second value:Score
Ranked list for query 3
[(18, 0.2310629000251795), (269, 0.20203937508890998), (48, 0.17161366480974374), (309, 0.11834119540684868), (274, 0.11369542630126248), (9, 0.10672211578270056), (288, 0.06392390014945243), (285, 0.05748075604644014), (129, 0.041304135383553695), (329, 0.036179097243736705)]

first value: Document, second value:Sco

first value: Document, second value:Score
Ranked list for query 28
[(238, 0.10042375699589422), (262, 0.08472498380823561), (35, 0.06447140147968945), (137, 0.057224984299744534), (293, 0.04866539184542707), (55, 0.04669390400220227), (160, 0.04453512658929848), (228, 0.044013531664910614), (66, 0.03898616942258848), (117, 0.038491115748716595)]

first value: Document, second value:Score
Ranked list for query 29
[(306, 0.14983750102299834), (282, 0.11944263617628582), (249, 0.10993244152436499), (41, 0.10528044782896565), (46, 0.09755228460798587), (227, 0.07102526096431548), (251, 0.07028102332701462), (261, 0.05709368260210791), (241, 0.05148329072132668), (111, 0.04794797492611261)]

first value: Document, second value:Score
Ranked list for query 30
[(149, 0.27898684632906334), (173, 0.25460733617310394), (267, 0.2422004278727682), (290, 0.1734307345571103), (127, 0.15838698756800232), (120, 0.12113906631052695), (35, 0.11927585332491454), (13, 0.09849566056502339), (49, 0.072074409